In [1]:
from ganime.model.moving_vae import MovingVAE
import tensorflow as tf
from ganime.data.base import load_dataset
import numpy as np

In [2]:
strategy = tf.distribute.MirroredStrategy()

2022-04-12 06:36:46.409364: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-12 06:36:48.950366: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14256 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:01:00.0, compute capability: 8.6
2022-04-12 06:36:48.951531: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14256 MB memory:  -> device: 1, name: NVIDIA RTX A4000, pci bus id: 0000:25:00.0, compute capability: 8.6
2022-04-12 06:36:48.952395: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 w

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [3]:
with strategy.scope():
    vae = MovingVAE(input_shape=(20, 64, 64, 1))

2022-04-12 06:36:50.355701: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [4]:
negloglik = lambda x, rv_x: -rv_x.log_prob(x)

with strategy.scope():
    vae.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
                loss=negloglik)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [5]:
# data = np.load("../../data/mnist_test_seq.npy")
# data.shape

# # We can see that data is of shape (window, n_samples, width, height)
# # But we want for keras something of shape (n_samples, window, width, height)
# data = np.moveaxis(data, 0, 1)
# # Also expand dimensions to have channels at the end (n_samples, window, width, height, channels)
# data = np.expand_dims(data, axis=-1)
# data.shape

# def _preprocess(sample):
#     image = tf.cast(sample, tf.float32) / 255.  # Scale to unit interval.
#     image = image < tf.random.uniform(tf.shape(image))   # Randomly binarize.
#     return image, image

# train_dataset = (tf.data.Dataset.from_tensor_slices(data[:9000])
#                  .map(_preprocess)
#                  .batch(256)
#                  .prefetch(tf.data.AUTOTUNE)
#                  .shuffle(int(10e3)))
# test_dataset = (tf.data.Dataset.from_tensor_slices(data[9000:])
#                  .map(_preprocess)
#                  .batch(256)
#                  .prefetch(tf.data.AUTOTUNE)
#                  .shuffle(int(10e3)))

In [6]:
# train_dataset, ds_info = load_dataset("my_dataset")

In [7]:
import ganime.data.my_dataset
import tensorflow_datasets as tfds

In [8]:
ds = tfds.load('my_dataset')

In [9]:
def normalize_img(sample):
  """Normalizes images: `uint8` -> `float32`."""
  video = sample["video"]
  label = sample["output"]
  # image = tf.cast(image, tf.float32) / 255.
  return video, label

In [10]:
import tensorflow as tf
dataset = ds["train"].map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).shuffle(int(10e3)).batch(256)#.prefetch(tf.data.AUTOTUNE)


In [11]:
vae.fit(dataset, epochs=2)

2022-04-12 06:36:52.515547: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:547] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.


Epoch 1/2
INFO:tensorflow:batch_all_reduce: 24 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 24 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2022-04-12 06:37:09.070905: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Feature: output (data type: string) is required but could not be found.
2022-04-12 06:37:09.070960: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Feature: output (data type: string) is required but could not be found.
2022-04-12 06:37:09.071012: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Feature: output (data type: string) is required but could not be found.
2022-04-12 06:37:09.071061: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Feature: output (data type: string) is required but could not be found.


InvalidArgumentError: Graph execution error:

4 root error(s) found.
  (0) INVALID_ARGUMENT:  Feature: output (data type: string) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[IteratorGetNextAsOptional]]
	 [[cond/pivot_t/_4/_165]]
  (1) INVALID_ARGUMENT:  Feature: output (data type: string) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[IteratorGetNextAsOptional]]
	 [[Adam/Adam/group_deps/_849]]
  (2) INVALID_ARGUMENT:  Feature: output (data type: string) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[IteratorGetNextAsOptional]]
  (3) CANCELLED:  Function was cancelled before it was started
0 successful operations.
1 derived errors ignored. [Op:__in